# Arqui de compus, SO y rercuperacion de fallas

## Disco y DBMS

Los recvord d elas bases de datos se almacenan en paginas de disco

A medica que se hace necesartios, las paginas son traidas a memoria principal (buffer)

## Paginas, disco y buffer

para trabjar con las tuplas de una realacion la bdd carga la pagina copn la tupla desde el discop

Para cargar esyas paginas la bdd reserva un espacio de RAM llamado buffer



## RAID matriz redundante de discos independientes

**Increementar el rendimiento y la fiablididad de SO**
a travfes de un arreglo de multiples disco que operan como uno solo

1. **Segmentacion de Datos (Data striping)**: Los datos se dividen en bloques disitbuidos en varios discos

2. **Redundancia de datos (Data mirroring)**: Los datos se duplican en varios discos para permitir **reconstruccion de datos** en caso de fallos

## RAID NIVELES

1. **RAID 0**: Data striping sin redundancia
2. **RAID 1**: **Espejado**, cada disco tiene una copia identica
3. **RAID 0 + 1**: Comnbinacion de Data striping y Espejado
4. **RAID 3**: **PARIDAD POR BIT** entrelazada por bit
5. **RAID 4**: **PARIDAD POR BLOQUE** entrelazada por bloque
6. **RAID 5**: **PARIDAD DISTRIBUIDA** como el NIvel 4, pero con **bloques de paridad distruibdos** en los discos

## HDD

CELDA UNIDAD MINIMA QUE ALMACENA BITS 

BLOQUE CONJUNTO DE PAGINAS LA UNIDAD MINIMA DE BORRADO

PLANO CONJUNTO DE BLOQUES FGESTIONADOS COMO UNA UNIDAD\\

IDE (SSD) chip de memoria que puede contener multipes planos


## ACID TRANSACTION


A: Atomicidad
C: Consistencia
I: Aislamiento
D: Durabilidad

* **ATomicity**: O se ejcutan todas las operecnaiosn o no se ejecuta ningu
* **Consistencia**: Cada transacciion preserva la consistencia de la BD (reglas de integridad, etc)
* **Isolation**: Cada transaccion se ejecuta como si fuera la unica en la BD, de forma aislada
* **Durabilidad**: Los cambios que hace cada transaccion son permanentes en el tiempo indepenediente de cualquiert tyiupo de de falla

## TRANSACCIONES

TRANSACION MANAGER

LOG MANAGER

RECOBERY MANAGER

**TRANSACTION MANAGER**: **ISOLATION Y CONSISTENCY**

**LOG Y RFECOVERY MANAGER**: **DURABILITY Y ATOMICITY**

## FAllas en transacciones

1. **Datos ERRONEOS**: Pueden ingresar en el sistema debido a errores humanos, fallas en la validacion de los datos o problemas en la integracion de sistemas externos SOLUCIONEs:

* **Restriciones de integridad**: IMPLEMENAR RESTRCICIONES A NIVBEL DE BDD COMO CALVE PRIMARIAS CLAVES FOREANEAS Y RESTRCIONES DE UNIDCIDAD Y VERIIFICACION

* **DATA CLEANING**: LIMPIEZA DE DATOS Y CORREGIR DAOS ERRONES O INCOMPLETOS, **NORMALIZACION, ELIMINACION DE DUPLICADOS y CORRECION DE FORMATOSINCORRECTOS**

2. **FALLAS EN EL ALMACEINAMIENO** FALAS FISICOS, lo que puede resultar en la perdiadfa de datos criticos. SOLUCIONEs:

* **RAID**: **RAID 1** y **RAID 5** para proteger los datos de fallas de disco. RAID 1 dupllicadop los datos en los dos disco durtos, mientras que raid 5 distribuye los datos junto con la paridad a traves de tres o mas discos

* **COPIAS REDUNDANTES**: Mantener copias de seguiroidad regulares en disposiyivos de almacenamientos separados


3. **CATASTROFES**: ERventos imprevbisdtos y graves como incendios, inundaciones terremostos, etc. SOLUCIONEs:

* **BACKUPS REGULARES**: Mantener copias de seguridad regulares en dispositivos de almacenamiento separados. (DOSINIODPS GEPGRAFICAMENE) ESTO asegura que si una ubicacion caga, se puede recuyperar de otros lados
* **REspaldos totales e incrementales**: Realizar copias de seguridad compeltnas de topda la bdd a **ibtervakis regulares**,
los cam,bios incrementalios guardan solo los cambios desde el ultimo respaldo

4. **Fallas del sistema**: Interrupciones en el funcionamiento normal del sistema de la BDD ya sea por fallos de software. hardware o erroreshumano. SOLUCIONEs:

**LOG Y RECOVERY MANAGER**

## Los logs comunes son:
* START T

* COMMIT T

* ABORT T

* T UPDATE

¿Cómo los usamos?, con Loggins.


## UNDO LOGGING

Los logs son:
* START T
  
* COMMIT T
  
* ABORT T

* T, X, t 

donde t es el valor antiguo de X


## Reglas undo loggin

Regla 1: si T modifica X, todos logs T, X, t deben
ser escritos antes que el valor X sea escrito en disco


Regla 2: si T hace commit, el log COMMIT T debe
ser escrito justo después de que todos los datos
modificados por T estén almacenados en disco


## Algoritmo para un Undo Logging

Procesamos el log desde el final hasta el principio:
* Si leo COMMIT T, marco T como realizada
* Si leo ABORT T, marco T como realizada
* Si leo T, X, t, debo restituir X := t en disco, si no
fue realizada.
* Si leo START T, lo ignoro

## NOnquiscent Checkpointing

SOn checkpoints que no piden reiniciar el sistema

Escribismos START CKPT (T1, ..., Tn) luego dque todos los datos de T1, ..., Tn esten en disco con commit escirbimnos END CKPT

## ALGIRUTNI DE UNDO LOGGING CON CHECKPOINTING

Empezamos desde el final, luego eliminamos todo loq ue empieza despues de START CKPT si es que antes encontramos un END CKPT



## REDO LOGGING

Los logs son:
* START T
* COMMIT T
* ABORT T
* T, X, v donde v es el nuevo valor de X
* T, END


## Reglas redo logging

Regla 1: Antes de modificar cualquier elemento X en
disco, es necesario que todos los logs estén
almacenados en disco, incluido el COMMIT



**AL REVES QUE UNDO LOGGING PRIMERO AL DISCOP LUEGO MODIFICAMOS**



## ALGORITMO PARA REDO LOGGING

Procesamos el log desde el principio hasta el final:

* Identificamos las **transacciones que hicieron commit sin hacer un END** SI HCIIERON END TODO OK
* HACEMOS SCAN DESDE EL PRINCIPIO
* SI LEO T X v,

    * SI T NO HIZO COMMIT NO HACER NADA
    * SI T HIZO COMMIT REESCRIBIR EL VALOR de v
* Para cada transaccion sin COMMIT, escrbir ABORT T


## USO DE CHECKPOINS EN REDO LLOGGING

* Escribimos un log START CKPT (T1, ..., Tn),
donde T1, ..., Tn son transacciones activas y sin
COMMIT
* Guardar todo el log en el disco
* Guardar en disco todo lo que haya hecho COMMIT
hasta ese punto; escribir en log END al finalizar
* Una vez hecho, escribir END CKPT



Revisar el log desde el final al inicio

* Si encontramos un END CKPT, debemos
retroceder hasta su su respectivo START CKPT
(T1, ..., Tn), y comenzar a hacer redo desde la
transacción más antigua entre T1, ..., Tn – las sin END

* No se hace redo de las transacciones con COMMIT
antes del START CKPT (T1, ..., Tn)
